# Normalizer

In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("normalizer").getOrCreate()

24/04/02 16:47:28 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface wlo1)
24/04/02 16:47:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/02 16:47:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.feature import Normalizer, VectorAssembler

In [3]:
cars = spark.read.csv("../0_data/Carros.csv", header=True, inferSchema=True, sep=";")
cars.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [6]:
vecasembler = VectorAssembler(inputCols=["Consumo", "Cilindros", "Cilindradas"], outputCol="vec")
cars_vect = vecasembler.transform(cars)
cars_vect.select("Consumo", "Cilindros", "Cilindradas", "vec").show(10)

+-------+---------+-----------+------------------+
|Consumo|Cilindros|Cilindradas|               vec|
+-------+---------+-----------+------------------+
|     21|        6|        160|  [21.0,6.0,160.0]|
|     21|        6|        160|  [21.0,6.0,160.0]|
|    228|        4|        108| [228.0,4.0,108.0]|
|    214|        6|        258| [214.0,6.0,258.0]|
|    187|        8|        360| [187.0,8.0,360.0]|
|    181|        6|        225| [181.0,6.0,225.0]|
|    143|        8|        360| [143.0,8.0,360.0]|
|    244|        4|       1467|[244.0,4.0,1467.0]|
|    228|        4|       1408|[228.0,4.0,1408.0]|
|    192|        6|       1676|[192.0,6.0,1676.0]|
+-------+---------+-----------+------------------+
only showing top 10 rows



In [9]:
norm = Normalizer(inputCol="vec", outputCol="vec_norm", p=1.0)
cars_norm = norm.transform(cars_vect)
cars_norm.select("Consumo", "Cilindros", "Cilindradas", "vec_norm").show(10, truncate=False)

+-------+---------+-----------+--------------------------------------------------------------+
|Consumo|Cilindros|Cilindradas|vec_norm                                                      |
+-------+---------+-----------+--------------------------------------------------------------+
|21     |6        |160        |[0.11229946524064172,0.03208556149732621,0.8556149732620321]  |
|21     |6        |160        |[0.11229946524064172,0.03208556149732621,0.8556149732620321]  |
|228    |4        |108        |[0.6705882352941176,0.011764705882352941,0.3176470588235294]  |
|214    |6        |258        |[0.4476987447698745,0.012552301255230125,0.5397489539748954]  |
|187    |8        |360        |[0.33693693693693694,0.014414414414414415,0.6486486486486487] |
|181    |6        |225        |[0.4393203883495146,0.014563106796116505,0.5461165048543689]  |
|143    |8        |360        |[0.27984344422700586,0.015655577299412915,0.7045009784735812] |
|244    |4        |1467       |[0.1422740524781341